# Notebook 00: Test Ollama Connection

This notebook helps diagnose and fix Ollama connection issues.

It will:
1. Check if Ollama is running
2. Test connectivity to the Ollama endpoint
3. Verify the embedding model is available
4. Provide troubleshooting steps if there are issues


In [1]:
# Import necessary modules
import sys
from pathlib import Path
import subprocess
import socket
import requests
import time

# Add the src directory to Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

print("Modules imported successfully!")


Modules imported successfully!


## Step 1: Check if Ollama Process is Running

First, let's check if Ollama is actually running on the system.


In [2]:
# Check if Ollama process is running
# We use 'pgrep' command to search for processes with 'ollama' in their name
# pgrep returns process IDs (PIDs) if found, or nothing if not found
try:
    # Run the pgrep command to find ollama processes
    # subprocess.run() executes a shell command and returns the result
    result = subprocess.run(
        ['pgrep', '-f', 'ollama'],  # -f means search in full command line, not just process name
        capture_output=True,        # Capture both stdout and stderr
        text=True                   # Return output as text string, not bytes
    )
    
    # Check if any processes were found (result.returncode == 0 means success)
    if result.returncode == 0 and result.stdout.strip():
        # Split the output by newlines to get individual process IDs
        pids = result.stdout.strip().split('\n')
        print(f"✅ Ollama is running!")
        print(f"   Found {len(pids)} Ollama process(es):")
        for pid in pids:
            print(f"   - Process ID: {pid}")
            
        # Try to get more info about the process using 'ps' command
        # This shows the full command that started the process
        for pid in pids:
            ps_result = subprocess.run(
                ['ps', '-p', pid, '-o', 'cmd='],  # Get command line for this PID
                capture_output=True,
                text=True
            )
            if ps_result.returncode == 0:
                print(f"   - Command: {ps_result.stdout.strip()}")
    else:
        print("❌ Ollama is NOT running")
        print("   No Ollama processes found on the system")
        
except FileNotFoundError:
    # pgrep might not be available on all systems
    print("⚠️  Could not check for Ollama process (pgrep not found)")
    print("   Trying alternative method...")
    
    # Alternative: try using 'ps' command directly
    try:
        ps_result = subprocess.run(
            ['ps', 'aux'],  # List all processes
            capture_output=True,
            text=True
        )
        if 'ollama' in ps_result.stdout.lower():
            print("✅ Found 'ollama' in process list")
            # Extract lines containing ollama
            for line in ps_result.stdout.split('\n'):
                if 'ollama' in line.lower():
                    print(f"   {line.strip()}")
        else:
            print("❌ Ollama is NOT running")
    except Exception as e:
        print(f"⚠️  Could not check processes: {e}")


✅ Ollama is running!
   Found 1 Ollama process(es):
   - Process ID: 18427
   - Command: /usr/local/bin/ollama serve


## Step 2: Test Network Connectivity to Ollama Port

Check if we can connect to port 11434 (Ollama's default port) on localhost.


In [3]:
# Test if we can connect to port 11434 on localhost
# This checks if something is listening on that port
def test_port_connection(host, port, timeout=2):
    """
    Test if we can connect to a specific host and port.
    
    Args:
        host: The hostname or IP address (e.g., 'localhost' or '127.0.0.1')
        port: The port number to test (e.g., 11434)
        timeout: How long to wait before giving up (in seconds)
    
    Returns:
        True if connection successful, False otherwise
    """
    try:
        # Create a socket object
        # socket.AF_INET means IPv4, socket.SOCK_STREAM means TCP
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        # Set a timeout so we don't wait forever
        sock.settimeout(timeout)
        
        # Try to connect to the host and port
        # This will raise an exception if connection fails
        result = sock.connect_ex((host, port))
        
        # Close the socket
        sock.close()
        
        # connect_ex returns 0 if connection successful, non-zero if failed
        return result == 0
        
    except Exception as e:
        # If anything goes wrong, return False
        print(f"   Error testing connection: {e}")
        return False

# Test localhost:11434
print("Testing connection to localhost:11434...")
host = "localhost"
port = 11434

if test_port_connection(host, port):
    print(f"✅ Port {port} is open and accepting connections on {host}")
else:
    print(f"❌ Cannot connect to {host}:{port}")
    print(f"   This means either:")
    print(f"   1. Ollama is not running")
    print(f"   2. Ollama is running on a different port")
    print(f"   3. Ollama is running on a different host")
    
# Also test 127.0.0.1 (sometimes different from localhost)
print(f"\nTesting connection to 127.0.0.1:{port}...")
if test_port_connection("127.0.0.1", port):
    print(f"✅ Port {port} is open on 127.0.0.1")
else:
    print(f"❌ Cannot connect to 127.0.0.1:{port}")


Testing connection to localhost:11434...
✅ Port 11434 is open and accepting connections on localhost

Testing connection to 127.0.0.1:11434...
✅ Port 11434 is open on 127.0.0.1


## Step 3: Test HTTP Connection to Ollama API

Try to actually make an HTTP request to Ollama's API to see if it responds.


In [4]:
# Test different possible Ollama endpoints
# Since you're on SSH, Ollama might be on a different address
possible_endpoints = [
    "http://localhost:11434",
    "http://127.0.0.1:11434",
    # If you're on a cluster, might need the actual hostname
    # Add more endpoints here if needed
]

def test_ollama_endpoint(base_url):
    """
    Test if we can reach Ollama's API at a given endpoint.
    
    Args:
        base_url: The base URL to test (e.g., "http://localhost:11434")
    
    Returns:
        Tuple of (success: bool, message: str, response_data: dict or None)
    """
    try:
        # Try to access Ollama's /api/tags endpoint
        # This endpoint lists available models and doesn't require any input
        url = f"{base_url}/api/tags"
        
        # Make a GET request with a short timeout
        # GET is used because we're just asking for information, not sending data
        response = requests.get(url, timeout=5)
        
        # Check if the request was successful (HTTP 200)
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            return True, "✅ Connection successful!", data
        else:
            # Got a response but it's an error
            return False, f"❌ Got HTTP {response.status_code}: {response.text}", None
            
    except requests.exceptions.ConnectionError as e:
        # Connection refused or couldn't reach the server
        return False, f"❌ Connection refused - Ollama not running at {base_url}", None
    except requests.exceptions.Timeout:
        # Request took too long
        return False, f"❌ Connection timeout - server not responding", None
    except Exception as e:
        # Some other error
        return False, f"❌ Error: {str(e)}", None

# Test each possible endpoint
print("Testing Ollama API endpoints...\n")
working_endpoint = None

for endpoint in possible_endpoints:
    print(f"Testing: {endpoint}")
    success, message, data = test_ollama_endpoint(endpoint)
    print(f"  {message}")
    
    if success and data:
        working_endpoint = endpoint
        # Show available models
        models = data.get('models', [])
        if models:
            print(f"  Available models ({len(models)}):")
            for model in models[:5]:  # Show first 5 models
                model_name = model.get('name', 'unknown')
                print(f"    - {model_name}")
            if len(models) > 5:
                print(f"    ... and {len(models) - 5} more")
        print()
        break
    print()

if working_endpoint:
    print(f"✅ Found working Ollama endpoint: {working_endpoint}")
    print(f"\n💡 Update your config to use: ollama_endpoint=\"{working_endpoint}\"")
else:
    print("❌ No working Ollama endpoint found")
    print("\n💡 Next steps:")
    print("   1. Make sure Ollama is installed")
    print("   2. Start Ollama with: ollama serve")
    print("   3. If running on a different host/port, update the endpoint in your config")


Testing Ollama API endpoints...

Testing: http://localhost:11434
  ✅ Connection successful!

✅ Found working Ollama endpoint: http://localhost:11434

💡 Update your config to use: ollama_endpoint="http://localhost:11434"


## Step 4: Test Embedding Model Availability

Check if the embedding model we need is available and can be used.


In [6]:
# Test if we can actually get an embedding from Ollama
# This is the real test - can we use the embedding model?

if not working_endpoint:
    print("⚠️  Skipping embedding test - no working endpoint found")
    print("   Please fix the connection issue first")
else:
    # Import the embedding function from our code
    from src.ollama_client import get_embedding
    
    # The model name we want to use
    model_name = "embeddinggemma"
    
    print(f"Testing embedding model: {model_name}")
    print(f"Using endpoint: {working_endpoint}\n")
    
    try:
        # Try to get an embedding for a simple test text
        # This is a small text to test if the model works
        test_text = "This is a test sentence for embedding."
        
        print(f"Test text: '{test_text}'")
        print("Calling Ollama embedding API...")
        
        # Call the embedding function
        # This will make a POST request to Ollama's /api/embeddings endpoint
        embedding, metadata = get_embedding(test_text, model_name, working_endpoint)
        
        # If we get here, it worked!
        print(f"✅ Embedding successful!")
        print(f"   Embedding dimension: {len(embedding)}")
        print(f"   First 5 values: {embedding[:5]}")
        print(f"   Model used: {metadata.get('model', 'unknown')}")
        
        if 'usage' in metadata:
            usage = metadata['usage']
            print(f"   Token usage: {usage}")
        
        print(f"\n✅ Everything is working! You can proceed with your notebook.")
        
    except Exception as e:
        print(f"❌ Embedding test failed: {e}")
        print(f"\nPossible issues:")
        print(f"   1. Model '{model_name}' is not installed")
        print(f"      Try: ollama pull {model_name}")
        print(f"   2. Model name is incorrect")
        print(f"      Check available models in Step 3 above")
        print(f"   3. Network or permission issues")


Testing embedding model: embeddinggemma
Using endpoint: http://localhost:11434

Test text: 'This is a test sentence for embedding.'
Calling Ollama embedding API...
✅ Embedding successful!
   Embedding dimension: 768
   First 5 values: [-0.1562882512807846, -0.0013628721935674548, 0.04643210023641586, 0.005612329114228487, -0.018335139378905296]
   Model used: None
   Token usage: {}

✅ Everything is working! You can proceed with your notebook.


## Troubleshooting Guide

If Ollama is not working, here are common solutions:

### 1. Start Ollama Service
If Ollama is not running, start it with:
```bash
ollama serve
```

### 2. Check if Ollama is Installed
```bash
which ollama
ollama --version
```

### 3. Pull the Required Model
If the model is not available:
```bash
ollama pull embeddinggemma
```

### 4. Check for Different Host/Port
If you're on a cluster or using SSH with port forwarding:
- Check what hostname/IP Ollama is actually running on
- Check if port forwarding is needed
- Update the `ollama_endpoint` in your config accordingly

### 5. Check Firewall/Security Groups
Make sure port 11434 is not blocked by firewall rules.


## Quick Fix: Start Ollama

If Ollama is not running, you can start it from this notebook. **Note:** You'll need to run this in a terminal separately since Ollama runs as a long-running service.


In [ ]:
# This cell shows you how to start Ollama
# Since Ollama needs to run continuously, you should start it in a separate terminal

print("To start Ollama, open a NEW terminal window and run:")
print("  ollama serve")
print()
print("Or if Ollama is not in your PATH, try:")
print("  ~/.local/bin/ollama serve")
print("  /usr/local/bin/ollama serve")
print()
print("After starting Ollama, come back to this notebook and re-run the cells above")
print("to verify the connection is working.")
print()
print("💡 TIP: You can run Ollama in the background with:")
print("  nohup ollama serve > ollama.log 2>&1 &")
print()
print("This will:")
print("  - Run Ollama in the background (nohup)")
print("  - Save output to ollama.log")
print("  - Continue running even if you close the terminal")
